In [4]:
import json
import xml.etree.ElementTree as ET
from datetime import datetime
from zoneinfo import ZoneInfo
import requests



class Train:

    def generate_image(self):
        
        api_key = 'e66ed4c930cffd68921f09b9d16496ff'
        if not api_key:
            raise RuntimeError("DB API Key not configured.")
        
        db_client_id = 'bed8ebafc8f7f59dd1b8aaf32dc2ec8f'
        if not db_client_id:
            raise RuntimeError("DB Client ID not configured.")
        
        try: 
            departures = self.get_departures(api_key, db_client_id)
            

        except Exception as e:
            raise RuntimeError("OpenWeatherMap request failure, please check logs.")

        # dimensions = device_config.get_resolution()
        # if device_config.get_config("orientation") == "vertical":
        #     dimensions = dimensions[::-1]

        # image = self.render_image(dimensions, "train_time.html", "train_time.css", departures)
        # if not image:
        #     raise RuntimeError("Failed to take screenshot, please check logs.")
        # return image

    
    def get_departures(self, api_key, db_client_id):
        DB_URL='https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/{evaNo}/{today}/{hour}'
        headers = {
            'DB-Client-Id': db_client_id,
            'DB-Api-Key': api_key,
            'accept': 'application/xml'
            }
        evaNo = '08011427'
        today = datetime.today().strftime("%y%m%d")
        hour = datetime.now(ZoneInfo("Europe/Berlin")).strftime("%H")
        
        response = requests.get(DB_URL.format(evaNo=evaNo, today=today, hour=hour), headers=headers)
        parsed_departures = self.parse_departures(response.content)
        return parsed_departures
    
    def parse_departures(self, xml_bytes):
        xml_string = xml_bytes.decode("utf-8")
        root = ET.fromstring(xml_string)
        departures = []

        for s in root.findall('s'):
            dp = s.find('dp')
            tl = s.find('tl')
            if dp is None or tl is None:
                continue

            # Zeitstempel in lesbares Format umwandeln (YYMMDDHHMM → datetime)
            pt = dp.attrib.get('pt')
            if not pt or len(pt) != 10:
                continue
            dt = datetime.strptime(pt, "%y%m%d%H%M")

            zugnummer = f"{tl.attrib.get('c', '')}{tl.attrib.get('n', '')}"
            gleis = dp.attrib.get('pp', '?')
            ppth = dp.attrib.get('ppth', '')
            ziel = ppth.split('|')[-1] if ppth else 'Unbekannt'

            departures.append({
                'zug': zugnummer,
                'abfahrt': dt.strftime('%H:%M'),
                'gleis': gleis,
                'ziel': ziel,
                'verspaetung': 0  # Kein Delay in XML enthalten
            })
        with open("data.js", "w", encoding="utf-8") as f:
            f.write("const departures = ")
            json.dump(departures, f, ensure_ascii=False, indent=2)
        return departures
    
train = Train()
train.generate_image()